In [ ]:
#header 값을 설정하지 않고 요청하기
import os
import requests


#마이너스의손
image_urls = [
    'https://image-comic.pstatic.net/webtoon/679568/105/20180914174738_ab27583b11eb33a170163cf46307fe13_IMAG01_1.jpg',
    'https://image-comic.pstatic.net/webtoon/679568/105/20180914174738_ab27583b11eb33a170163cf46307fe13_IMAG01_2.jpg',
    'https://image-comic.pstatic.net/webtoon/679568/105/20180914174738_ab27583b11eb33a170163cf46307fe13_IMAG01_3.jpg'
]
for image_url in image_urls:
    response = requests.get(image_url)
    image_data = response.content
    filename = os.path.basename(image_url)
    with open(filename, 'wb') as f:
        print('writing to {} ({} bytes)'.format(filename, len(image_data)))
        f.write(image_data)

In [19]:
# header 값을 설정하여 요청하기
import os
import requests

# https://comic.naver.com/webtoon/detail.nhn?titleId=679568&no=105&weekday=mon - 마이너스의손
image_urls = [
    'https://image-comic.pstatic.net/webtoon/679568/105/20180914174738_ab27583b11eb33a170163cf46307fe13_IMAG01_1.jpg',
    'https://image-comic.pstatic.net/webtoon/679568/105/20180914174738_ab27583b11eb33a170163cf46307fe13_IMAG01_2.jpg',
    'https://image-comic.pstatic.net/webtoon/679568/105/20180914174738_ab27583b11eb33a170163cf46307fe13_IMAG01_3.jpg'
]
for image_url in image_urls:
    headers = {
        'Referer': 'https://comic.naver.com/webtoon/detail.nhn?titleId=679568&no=105&weekday=mon'    
    }
    response = requests.get(image_url, headers=headers)
    image_data = response.content
    #print(type(response),type(image_data))  #<class 'requests.models.Response'> <class 'bytes'>
    filename = os.path.basename(image_url)
    print(filename)
    with open(filename, 'wb') as f:
        print('writing to {} ({} bytes)'.format(filename, len(image_data)))
        f.write(image_data)

20180914174738_ab27583b11eb33a170163cf46307fe13_IMAG01_1.jpg
writing to 20180914174738_ab27583b11eb33a170163cf46307fe13_IMAG01_1.jpg (97370 bytes)
20180914174738_ab27583b11eb33a170163cf46307fe13_IMAG01_2.jpg
writing to 20180914174738_ab27583b11eb33a170163cf46307fe13_IMAG01_2.jpg (123451 bytes)
20180914174738_ab27583b11eb33a170163cf46307fe13_IMAG01_3.jpg
writing to 20180914174738_ab27583b11eb33a170163cf46307fe13_IMAG01_3.jpg (131449 bytes)


In [34]:
#title(제목), 회차, url 을 입력 받아서 저장하는 함수 정의
import os
import requests
from bs4 import BeautifulSoup

def write_image(title,seq, url):
        
    #url = 'https://comic.naver.com/webtoon/detail.nhn?titleId=749279&no=4&weekday=mon'
    response = requests.get(url)
    #print(response.status_code)
    soup = BeautifulSoup(response.text, 'html.parser')

    image_urls = []
    for img in soup.select('img[src$=".jpg"]'):
        #print(img['src'])
        image_urls.append(img['src'])
        
    dir_name = 'img/'+title + '_' + seq
    if not os.path.isdir(dir_name):
        os.mkdir(dir_name)

    for image_url in image_urls:
        headers = {
            'Referer': url
        }
        #print(image_url)
        res = requests.get(image_url, headers=headers)
        #print(res.status_code)
        image_data = res.content
        file_name = os.path.basename(image_url)

        with open(dir_name+'/'+file_name,'wb') as f:
            print('image file name {} ({} bytes)'.format(file_name,len(image_data)))
            f.write(image_data)
            
#write_image('대만에서','1화','https://comic.naver.com/webtoon/detail.nhn?titleId=749279&no=4&weekday=mon')

In [35]:
#네이버 웹툰 메인 페이지에서 만화 제목과 링크를 추출해서 list에 저장한다
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

url = 'https://comic.naver.com/index.nhn'

response = requests.get(url)
#print(response.status_code)
soup = BeautifulSoup(response.text, 'html.parser')

#a_tag_list = soup.select('#genreRecommand div.genreRecomInfo2 > h6 > a')
a_tag_list = soup.select('.genreRecomInfo2 h6 a')
#print(a_tag_list)

webtoon_list = []
for a_tag in a_tag_list:
    #if a_tag['href'] != '#none':
    #print(a_tag)
    link = urljoin(url,a_tag['href'])
    title = a_tag.text.strip()
    #print(title, link)
    webtoon_dict = {'title':title, 'link':link}
    webtoon_list.append(webtoon_dict)
    webtoon_dict = {}

print(len(webtoon_list))        
print(webtoon_list)    

6
[{'title': '히어로메이커', 'link': 'https://comic.naver.com/webtoon/list.nhn?titleId=21815'}, {'title': '대만에서, 너와 하루', 'link': 'https://comic.naver.com/webtoon/list.nhn?titleId=749279'}, {'title': '야자괴담', 'link': 'https://comic.naver.com/webtoon/list.nhn?titleId=743025'}, {'title': '중독연구소', 'link': 'https://comic.naver.com/webtoon/list.nhn?titleId=740132'}, {'title': '결혼생활 그림일기', 'link': 'https://comic.naver.com/webtoon/list.nhn?titleId=741891'}, {'title': '귀전구담', 'link': 'https://comic.naver.com/webtoon/list.nhn?titleId=694946'}]


In [39]:
for webtoon_link in webtoon_list[0:2]:
    link = webtoon_link['link']
    response = requests.get(link)
    #print(response.status_code)
    soup = BeautifulSoup(response.text, 'html.parser')
    title = webtoon_link['title']
    a_tag_list = soup.select('td.title > a')
    for a_tag in a_tag_list:
        #print(a_tag)
        link2 = urljoin(link, a_tag['href'])
        seq = a_tag.text
        
        # 글제목에 '화' 라는 문자열이 있으면
        if '화' in seq:
            my_index = seq.index('화')
            #print(my_index)
            # '화' 문자열의 인덱스 기준으로 앞의 문자열을 슬라이싱 한다
            seq = seq[:my_index+1]
        print('------',title, seq)
        write_image(title,seq,link2)

------ 히어로메이커 재심사
image file name 20200717181605_906e36712889a5a922e1326491a1128f_IMAG01_1.jpg (145036 bytes)
image file name 20200717181605_906e36712889a5a922e1326491a1128f_IMAG01_2.jpg (154780 bytes)
image file name 20200717181605_906e36712889a5a922e1326491a1128f_IMAG01_3.jpg (204531 bytes)
image file name 20200717181605_906e36712889a5a922e1326491a1128f_IMAG01_4.jpg (155246 bytes)
image file name 20200717181605_906e36712889a5a922e1326491a1128f_IMAG01_5.jpg (179588 bytes)
image file name 20200717181605_906e36712889a5a922e1326491a1128f_IMAG01_6.jpg (147698 bytes)
image file name 20200717181605_906e36712889a5a922e1326491a1128f_IMAG01_7.jpg (198055 bytes)
image file name 20200717181605_906e36712889a5a922e1326491a1128f_IMAG01_8.jpg (179605 bytes)
image file name 20200717181605_906e36712889a5a922e1326491a1128f_IMAG01_9.jpg (142693 bytes)
image file name 20200717181605_906e36712889a5a922e1326491a1128f_IMAG01_10.jpg (148089 bytes)
image file name thumbnail_202x120_c94a9966-caa9-472f-8aee-de3

image file name 20200612174324_3d4970a057ba3939e0920272f0b30b0c_IMAG01_1.jpg (130033 bytes)
image file name 20200612174324_3d4970a057ba3939e0920272f0b30b0c_IMAG01_2.jpg (128908 bytes)
image file name 20200612174324_3d4970a057ba3939e0920272f0b30b0c_IMAG01_3.jpg (156310 bytes)
image file name 20200612174324_3d4970a057ba3939e0920272f0b30b0c_IMAG01_4.jpg (282786 bytes)
image file name 20200612174324_3d4970a057ba3939e0920272f0b30b0c_IMAG01_5.jpg (238668 bytes)
image file name 20200612174324_3d4970a057ba3939e0920272f0b30b0c_IMAG01_6.jpg (204422 bytes)
image file name 20200612174324_3d4970a057ba3939e0920272f0b30b0c_IMAG01_7.jpg (110443 bytes)
image file name 20200612174324_3d4970a057ba3939e0920272f0b30b0c_IMAG01_8.jpg (130637 bytes)
image file name 20200612174324_3d4970a057ba3939e0920272f0b30b0c_IMAG01_9.jpg (190845 bytes)
image file name 20200612174324_3d4970a057ba3939e0920272f0b30b0c_IMAG01_10.jpg (54412 bytes)
image file name thumbnail_202x120_0a01358d-9f02-4ef5-a6b7-8e9dd9ab13b5.jpg (3397

image file name thumbnail_202x120_0a01358d-9f02-4ef5-a6b7-8e9dd9ab13b5.jpg (33970 bytes)
image file name thumbnail_202x120_6d498f53-0188-43d6-b32c-a4fbcf79b4ec.jpg (25433 bytes)
image file name thumbnail_202x120_cefd1d1d-d814-4665-b186-5cb9a63350bb.jpg (13981 bytes)
image file name thumbnail_IMAG10_261e36e7-8e8f-4640-bcf8-31232c5279b4.jpg (6934 bytes)
image file name title_thumbnail_20140829190825_t83x90.jpg (5514 bytes)
------ 대만에서, 너와 하루 04화
image file name thumbnail_IMAG04_cdcdf98d-00fa-4470-9147-3486dde689ab.jpg (28014 bytes)
image file name 20200716152201_0af758a65232bf334a14efccfb82d9a4_IMAG01_1.jpg (183439 bytes)
image file name 20200716152201_0af758a65232bf334a14efccfb82d9a4_IMAG01_2.jpg (92084 bytes)
image file name 20200716152201_0af758a65232bf334a14efccfb82d9a4_IMAG01_3.jpg (142515 bytes)
image file name 20200716152201_0af758a65232bf334a14efccfb82d9a4_IMAG01_4.jpg (213724 bytes)
image file name 20200716152201_0af758a65232bf334a14efccfb82d9a4_IMAG01_5.jpg (199297 bytes)
image

image file name 20200625165235_201fab3123ccef35118b1d5ef794f66a_IMAG01_6.jpg (142536 bytes)
image file name 20200625165235_201fab3123ccef35118b1d5ef794f66a_IMAG01_7.jpg (180859 bytes)
image file name 20200625165235_201fab3123ccef35118b1d5ef794f66a_IMAG01_8.jpg (214191 bytes)
image file name 20200625165235_201fab3123ccef35118b1d5ef794f66a_IMAG01_9.jpg (183578 bytes)
image file name 20200625165235_201fab3123ccef35118b1d5ef794f66a_IMAG01_10.jpg (162818 bytes)
image file name 20200625165235_201fab3123ccef35118b1d5ef794f66a_IMAG01_11.jpg (165109 bytes)
image file name 20200625165235_201fab3123ccef35118b1d5ef794f66a_IMAG01_12.jpg (153233 bytes)
image file name 20200625165235_201fab3123ccef35118b1d5ef794f66a_IMAG01_13.jpg (176517 bytes)
image file name 20200625165235_201fab3123ccef35118b1d5ef794f66a_IMAG01_14.jpg (110004 bytes)
image file name 20200625165235_201fab3123ccef35118b1d5ef794f66a_IMAG01_15.jpg (258187 bytes)
image file name 20200625165235_201fab3123ccef35118b1d5ef794f66a_IMAG01_16.